In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
import time

load_dotenv()

True

api key 설정, 헬퍼 함수 정의

In [2]:
api_key = os.environ.get("OpenAI_API_KEY")

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

Assistant 생성

In [3]:
client = OpenAI(api_key=api_key)
assistant = client.beta.assistants.create(
    name="Recipe Bot",
    instructions="""
        You are a culinary assistant specializing in providing tailored recipes. Based on the given inputs:

        Selected ingredients: A list of ingredients chosen by the user.
        Preferred cooking style: A brief description of the desired flavor or type of dish (e.g., spicy, sweet, or savory).

        Your task is to generate three creative and practical recipes using the provided ingredients while considering the user's cooking preferences. Each recipe should include:

        - A title describing the dish.
        - A brief summary of the dish.
        - A list of ingredients (including the selected ones).
        - Step-by-step cooking instructions.

        If the user does not specify a cooking style, default to a general, versatile approach. Aim to inspire and delight the user with your suggestions.

        If the selected ingredients are insufficient for the recipe, suggest additional ingredients and explain why they are necessary.
        
        Your responses should be in Korean.
    """,
    model="gpt-4o-mini"
)

show_json(assistant)
ASSISTANT_ID = assistant.id

{'id': 'asst_7YMsCAsFx2DIqWpVKvP5u9RF',
 'created_at': 1732084660,
 'description': None,
 'instructions': "\n        You are a culinary assistant specializing in providing tailored recipes. Based on the given inputs:\n\n        Selected ingredients: A list of ingredients chosen by the user.\n        Preferred cooking style: A brief description of the desired flavor or type of dish (e.g., spicy, sweet, or savory).\n\n        Your task is to generate three creative and practical recipes using the provided ingredients while considering the user's cooking preferences. Each recipe should include:\n\n        - A title describing the dish.\n        - A brief summary of the dish.\n        - A list of ingredients (including the selected ones).\n        - Step-by-step cooking instructions.\n\n        If the user does not specify a cooking style, default to a general, versatile approach. Aim to inspire and delight the user with your suggestions.\n\n        If the selected ingredients are insuffic

Thread 생성
- 스레드에 메시지 추가
- 스레드 실행(run)
- 스레드 상태(대기, 작업, 완료 등) 확인
- 스레드 최신 메시지 조회

In [11]:
def create_thread():
    thread = client.beta.threads.create()
    return thread

thread = create_thread()
show_json(thread)
THREAD_ID = thread.id

{'id': 'thread_ROKjd6qscvMncYLp0TENeckA',
 'created_at': 1732084774,
 'metadata': {},
 'object': 'thread',
 'tool_resources': {'code_interpreter': None, 'file_search': None}}

In [4]:
def wait_on_run(run, thread_id):
    # run이 완료될 때까지 대기
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run.id
        )
        time.sleep(0.5)  # 서버 부하를 줄이기 위한 API 요청 사이 대기 시간
    return run

def submit_message(assistant_id, thread_id, user_input):
    ingredients, cooking_style = user_input[0], user_input[1]
    
    message_content = f"""
    Here are the user's inputs:
    - Selected ingredients: {ingredients}
    - Preferred cooking style: {cooking_style}
    
    Please generate three recipes considering the above inputs.
    """
    
    client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=message_content
    )

    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id
    )
    return run

def get_response(thread_id):
    return client.beta.threads.messages.list(thread_id=thread_id, order="asc")

def print_message(response):
    for res in response:
        print(f"[{res.role.upper()}]\n{res.content[0].text.value}\n")
    print("-" * 50)

def ask(assistant_id, thread_id, user_input):
    run = submit_message(assistant_id, thread_id, user_input)
    run = wait_on_run(run, thread_id)
    response_data = get_response(thread_id).data
    if response_data:
        # -2 : [USER] + [ASSISTANT]
        # -1 : [ASSISTANT] only
        print_message(response_data[-2:])
    else:
        print("응답이 없습니다.")
    
    return run

In [5]:
ingredients = "양파, 감자, 파스타면, 양송이, 간장, 김치, 대파, 삼겹살, 간장, 토마토, 마늘, 굴소스, 두부, 계란"
cooking_style = "약간 맵게 해줘"

In [ ]:
# user_input = [ingredients, cooking_style]
# run = ask(ASSISTANT_ID, THREAD_ID, user_input)

[ASSISTANT]
사용자가 선택한 재료와 약간 매운 요리 스타일을 바탕으로 세 가지 레시피를 준비했습니다. 맛있게 만들어 보세요!

### 레시피 1: 매콤한 삼겹살 야채 볶음

**요약:** 삼겹살과 다양한 야채를 매콤하게 볶아 밥과 함께 즐기는 요리입니다.

**재료:**
- 삼겹살 300g
- 양파 1개
- 감자 1개
- 양송이 150g
- 대파 1대
- 간장 2큰술
- 굴소스 1큰술
- 고춧가루 1큰술 (추가)
- 마늘 2쪽 (다진 것)
- 올리브유 2큰술
- 소금, 후추 약간

**조리 과정:**
1. 삼겹살은 한 입 크기로 썰고, 양파와 감자는 채썰고, 양송이는 슬라이스 합니다. 대파는 어슷썰기 합니다.
2. 팬에 올리브유를 두르고 삼겹살을 넣어 노릇하게 볶습니다.
3. 삼겹살이 익으면 다진 마늘과 양파를 넣고 볶다가, 감자와 양송이를 추가합니다.
4. 간장과 굴소스를 넣고 고춧가루를 넣어 잘 섞은 후, 재료들이 골고루 익도록 볶습니다.
5. 마지막으로 대파를 넣고 소금과 후추로 간을 맞추고 잘 섞어줍니다.
6. 식사로 제공합니다.

---

### 레시피 2: 매운 김치 두부 스튜

**요약:** 김치와 두부를 이용해 매콤하고 깊은 맛의 스튜를 만드는 조리법입니다.

**재료:**
- 두부 1모
- 김치 1컵
- 양파 1개
- 대파 1대
- 마늘 2쪽 (다진 것)
- 간장 2큰술
- 고추장 1큰술
- 물 2컵
- 후춧가루 약간

**조리 과정:**
1. 두부는 먹기 좋은 크기로 자르고, 양파는 채썰고 대파는 어슷썰어 줍니다.
2. 냄비에 다진 마늘을 넣고 향이 나게 볶은 후, 양파와 김치를 추가합니다.
3. 김치가 익을 때까지 볶은 후, 물을 부어 끓입니다.
4. 끓기 시작하면 두부를 넣고 간장, 고추장을 추가한 후 중약불로 15-20분 끓입니다.
5. 대파를 넣고 후춧가루로 간을 맞춘 후, 그릇에 담아 제공합니다.

---

### 레시피 3: 매콤한 토마토 파스타

**요약:** 신선한 토마토와 양송이를 매콤하게 조리한 파스타 요리입니다.


file_search 기능 추가

In [6]:
file = client.files.create(
    file=open("../crawling/recipes.json", "rb"),
    purpose="assistants"
)

In [7]:
assistant = client.beta.assistants.create(
    name="Recipe Bot",
    model="gpt-4o-mini",
    tools=[{"type": "file_search"}],
    instructions="""
        You are a culinary assistant specializing in providing tailored recipes. Based on the given inputs:

        Selected ingredients: A list of ingredients chosen by the user.
        Preferred cooking style: A brief description of the desired flavor or type of dish (e.g., spicy, sweet, or savory).

        Your task is to generate three creative and practical recipes using the provided ingredients while considering the user's cooking preferences. Each recipe should include:

        - A title describing the dish.
        - A brief summary of the dish.
        - A list of ingredients (including the selected ones).
        - Step-by-step cooking instructions.

        If the user does not specify a cooking style, default to a general, versatile approach. Aim to inspire and delight the user with your suggestions.

        In addition to your culinary expertise, you can retrieve information from the uploaded documents to enhance your suggestions or answer user queries. Use this knowledge to provide more accurate, detailed, and contextually relevant recipes or suggestions.

        If the selected ingredients are insufficient for the recipe, suggest additional ingredients and explain why they are necessary.

        Your responses should be in Korean.
    """
)

show_json(assistant)

{'id': 'asst_YCWA6UtK36YnWaSAZlV2jMH7',
 'created_at': 1732084722,
 'description': None,
 'instructions': "\n        You are a culinary assistant specializing in providing tailored recipes. Based on the given inputs:\n\n        Selected ingredients: A list of ingredients chosen by the user.\n        Preferred cooking style: A brief description of the desired flavor or type of dish (e.g., spicy, sweet, or savory).\n\n        Your task is to generate three creative and practical recipes using the provided ingredients while considering the user's cooking preferences. Each recipe should include:\n\n        - A title describing the dish.\n        - A brief summary of the dish.\n        - A list of ingredients (including the selected ones).\n        - Step-by-step cooking instructions.\n\n        If the user does not specify a cooking style, default to a general, versatile approach. Aim to inspire and delight the user with your suggestions.\n\n        In addition to your culinary expertise, 

In [8]:
# Create a vector store called "RECIPE"
vector_store = client.beta.vector_stores.create(name="RECIPE")
 
# Ready the files for upload to OpenAI
file_paths = ["../crawling/recipes.json"]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

show_json(vector_store)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


{'id': 'vs_bZbGOHl01SSYVHwZGz0w6OuI',
 'created_at': 1732084742,
 'file_counts': {'cancelled': 0,
  'completed': 0,
  'failed': 0,
  'in_progress': 0,
  'total': 0},
 'last_active_at': 1732084742,
 'metadata': {},
 'name': 'RECIPE',
 'object': 'vector_store',
 'status': 'completed',
 'usage_bytes': 0,
 'expires_after': None,
 'expires_at': None}

In [9]:
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
)
show_json(assistant)

{'id': 'asst_YCWA6UtK36YnWaSAZlV2jMH7',
 'created_at': 1732084722,
 'description': None,
 'instructions': "\n        You are a culinary assistant specializing in providing tailored recipes. Based on the given inputs:\n\n        Selected ingredients: A list of ingredients chosen by the user.\n        Preferred cooking style: A brief description of the desired flavor or type of dish (e.g., spicy, sweet, or savory).\n\n        Your task is to generate three creative and practical recipes using the provided ingredients while considering the user's cooking preferences. Each recipe should include:\n\n        - A title describing the dish.\n        - A brief summary of the dish.\n        - A list of ingredients (including the selected ones).\n        - Step-by-step cooking instructions.\n\n        If the user does not specify a cooking style, default to a general, versatile approach. Aim to inspire and delight the user with your suggestions.\n\n        In addition to your culinary expertise, 

In [12]:
user_input = [ingredients, cooking_style]
run = ask(assistant.id, thread.id, user_input)

[USER]

    Here are the user's inputs:
    - Selected ingredients: 양파, 감자, 파스타면, 양송이, 간장, 김치, 대파, 삼겹살, 간장, 토마토, 마늘, 굴소스, 두부, 계란
    - Preferred cooking style: 약간 맵게 해줘
    
    Please generate three recipes considering the above inputs.
    

[ASSISTANT]
아래는 주어진 재료와 선호하는 조리 스타일인 '약간 맵게'를 반영한 세 가지 요리 레시피입니다.

### 1. 매콤한 삼겹살 감자 파스타

**요약:** 부드럽고 매콤한 양념으로 볶은 삼겹살과 감자를 파스타 면에 버무린 이색적인 요리입니다.

**재료:**
- 양파 1개, 채썰기
- 감자 1개, 깍둑썰기
- 파스타면 200g
- 삼겹살 200g, 잘게 썰기
- 마늘 2쪽, 다지기
- 간장 2큰술
- 굴소스 1큰술
- 대파 1대, 송송 썰기
- 고추가루 1큰술 (약간 맵게 하고 싶다면 추가)
- 후추 약간
- 올리브유 또는 식용유 약간

**조리 방법:**
1. 파스타 면을 끓는 물에 삶아 알단테로 준비한 후 물기를 빼고 놓아둡니다.
2. 팬에 기름을 두르고 다진 마늘과 양파를 넣어 볶다가, 향이 올라오면 삼겹살을 넣고 중불에서 볶습니다.
3. 삼겹살이 익으면 깍둑썰기한 감자를 넣고 함께 볶아줍니다.
4. 감자가 어느 정도 익으면 간장, 굴소스, 고추가루, 후추를 넣어 양념을 잘 섞어줍니다.
5. 마지막으로 삶은 파스타 면과 대파를 넣고 잘 섞어가며 볶아줍니다.
6. 접시에 담고, 원하시면 추가로 고추를 올려서 서빙합니다.

---

### 2. 매콤 김치 두부찌개

**요약:** 깊은 맛을 자랑하는 두부와 김치를 활용한 매콤한 찌개입니다.

**재료:**
- 김치 200g
- 두부 1모, 큐브 모양으로 잘라놓기
- 양파 1개, 채썰기
- 대파 1대, 송송 썰기
- 마늘 2쪽, 다져놓기
- 간장 2큰술
- 고추가